In [1]:
%pip install videodb


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

In [4]:
import videodb

conn = videodb.connect()
coll = conn.get_collection()

In [5]:
video = coll.upload(url="https://youtu.be/kcW4ABcY3zI")
video.play()

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/ac445699-1401-4470-b71f-3617b3678a30.m3u8'

In [6]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:24<00:00,  4.03it/s]


In [7]:
from videodb import SearchType

result = video.search(query='education', search_type=SearchType.keyword)

In [13]:
print(result.shots)


[Shot(video_id=m-893892de-31bb-43b2-b5c1-aee290710049, video_title=A well educated mind vs a well formed mind: Dr. Shashi Tharoor at TEDxGateway 2013, start=12.606, end=13.054, text=education, search_score=1, stream_url=None, player_url=None), Shot(video_id=m-893892de-31bb-43b2-b5c1-aee290710049, video_title=A well educated mind vs a well formed mind: Dr. Shashi Tharoor at TEDxGateway 2013, start=13.526, end=13.902, text=education, search_score=1, stream_url=None, player_url=None), Shot(video_id=m-893892de-31bb-43b2-b5c1-aee290710049, video_title=A well educated mind vs a well formed mind: Dr. Shashi Tharoor at TEDxGateway 2013, start=48.506, end=48.858, text=education, search_score=1, stream_url=None, player_url=None), Shot(video_id=m-893892de-31bb-43b2-b5c1-aee290710049, video_title=A well educated mind vs a well formed mind: Dr. Shashi Tharoor at TEDxGateway 2013, start=51.066, end=51.658, text=education, search_score=1, stream_url=None, player_url=None), Shot(video_id=m-893892de-31

In [17]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset, TextAsset, ImageAsset
from videodb import play_stream

timeline = Timeline(conn)

seeker=0
counter=1
padding=0.6

for shot in result.shots:
    video_asset = VideoAsset(asset_id=shot.video_id, start= shot.start-padding, end=shot.end+padding)

    duration = shot.end - shot.start + 2*padding
    text_asset = TextAsset(
    text=f"{counter}",
    duration= duration
    )
    timeline.add_inline(asset=video_asset)
    timeline.add_overlay(asset=text_asset, start = seeker)

    seeker += duration 
    counter += 1


stream_url = timeline.generate_stream()
print(f"https://console.videodb.io/player?={stream_url}")
play_stream(stream_url)

start :  12.606  duration   1.6480000000000004  index  1
start :  13.526  duration   1.5759999999999994  index  2
start :  48.506  duration   1.5519999999999967  index  3
start :  51.066  duration   1.7919999999999987  index  4
start :  171.208  duration   1.6879999999999995  index  5
start :  190.648  duration   1.567999999999995  index  6
start :  203.294  duration   1.7839999999999747  index  7
start :  224.414  duration   1.5200000000000216  index  8
start :  239.182  duration   1.7720000000000027  index  9
start :  243.75  duration   1.7560000000000116  index  10
start :  266.49  duration   1.7639999999999645  index  11
start :  314.116  duration   1.5520000000000322  index  12
start :  353.41  duration   1.5759999999999763  index  13
start :  364.114  duration   1.6159999999999968  index  14
start :  384.93  duration   1.5759999999999763  index  15
start :  394.818  duration   1.75600000000004  index  16
start :  510.882  duration   1.7520000000000209  index  17
https://console.v

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/72ce8d28-17de-414d-88b1-a455156a62ea.m3u8'